In [1]:
import gurobipy as gp
from gurobipy import GRB

env = gp.Env(empty=True)
env.setParam("OutputFlag",1)
env.start()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-31


<gurobipy.Env, No parameter changes>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [3]:
# read files
# bus, generator, branch
df_bus = pd.read_csv("case30_bus.csv", header=None)
df_generator = pd.read_csv("case30_gen.csv", header=None)
df_branch = pd.read_csv("case30_branch.csv", header=None)

In [4]:
df_bus.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,3,0.0,0.0,0,0.00,1,1,0,135,1,1.05,0.95
1,2,2,21.7,12.7,0,0.00,1,1,0,135,1,1.10,0.95
2,3,1,2.4,1.2,0,0.00,1,1,0,135,1,1.05,0.95
3,4,1,7.6,1.6,0,0.00,1,1,0,135,1,1.05,0.95
4,5,1,0.0,0.0,0,0.19,1,1,0,135,1,1.05,0.95


In [5]:
df_generator.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,1,23.54,0,150.0,-20,1,100,1,80,0,...,0,0,0,0,0,0,0,0,0,0
1,2,60.97,0,60.0,-20,1,100,1,80,0,...,0,0,0,0,0,0,0,0,0,0
2,22,21.59,0,62.5,-15,1,100,1,50,0,...,0,0,0,0,0,0,0,0,0,0
3,27,26.91,0,48.7,-15,1,100,1,55,0,...,0,0,0,0,0,0,0,0,0,0
4,23,19.20,0,40.0,-10,1,100,1,30,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df_branch.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,2,0.02,0.06,0.03,130,130,130,0,0,1,-360,360
1,1,3,0.05,0.19,0.02,130,130,130,0,0,1,-360,360
2,2,4,0.06,0.17,0.02,65,65,65,0,0,1,-360,360
3,3,4,0.01,0.04,0.00,130,130,130,0,0,1,-360,360
4,2,5,0.05,0.20,0.02,130,130,130,0,0,1,-360,360


In [7]:
df_bus = df_bus.drop(3, axis=1)
df_bus = df_bus.drop(4, axis=1)
df_bus = df_bus.drop(5, axis=1)
df_bus = df_bus.drop(7, axis=1)
df_bus = df_bus.drop(8, axis=1)
df_bus = df_bus.drop(9, axis=1)
df_bus = df_bus.drop(10, axis=1)
df_bus = df_bus.rename(
    columns={0: "bus_i", 1: "type", 2: "Pd", 6: "area", 11: "Vmax", 12: "Vmin"}
)
df_bus.head()

,bus_i,type,Pd,area,Vmax,Vmin
0,1,3,0.0,1,1.05,0.95
1,2,2,21.7,1,1.10,0.95
2,3,1,2.4,1,1.05,0.95
3,4,1,7.6,1,1.05,0.95
4,5,1,0.0,1,1.05,0.95


In [8]:
df_generator = df_generator.drop(2, axis=1)
df_generator = df_generator.drop(3, axis=1)
df_generator = df_generator.drop(4, axis=1)
df_generator = df_generator.drop(6, axis=1)
df_generator = df_generator.drop(7, axis=1)
for i in range(10, 21):
    df_generator = df_generator.drop(i, axis=1)
df_generator = df_generator.rename(
    columns={0: "bus", 1: "Pg", 5: "Vg", 8: "Pmax", 9: "Pmin"}
)
df_generator.head()

,bus,Pg,Vg,Pmax,Pmin
0,1,23.54,1,80,0
1,2,60.97,1,80,0
2,22,21.59,1,50,0
3,27,26.91,1,55,0
4,23,19.20,1,30,0


In [9]:
for i in range(2, 4):
    df_branch = df_branch.drop(i, axis=1)

for i in range(5, 7):
    df_branch = df_branch.drop(i, axis=1)

for i in range(8, 10):
    df_branch = df_branch.drop(i, axis=1)

df_branch = df_branch.drop(10, axis=1)
df_branch = df_branch.rename(
    columns={
        0: "fbus",
        1: "tbus",
        4: "susceptance",
        7: "thermal",
        9: "angle",
        11: "angmin",
        12: "angmax",
    }
)
df_branch.head()

,fbus,tbus,susceptance,thermal,angmin,angmax
0,1,2,0.03,130,-360,360
1,1,3,0.02,130,-360,360
2,2,4,0.02,65,-360,360
3,3,4,0.00,130,-360,360
4,2,5,0.02,130,-360,360


In [10]:
#################################################
####                   Sets                  ####
#################################################
# set of subareas
A_e = range(1, 1 + df_bus["area"].nunique())
# set of days in a week
W = range(1, 8)
# set of hours in a day
H = range(1, 25)

# electrical components
# set of buses
B = range(1, 1 + df_bus["bus_i"].nunique())
# set of loads
D_wh = {}
for index, row in df_bus.iterrows():
    D_wh[int(row["bus_i"])] = row["Pd"]
# set of lines
L = []
for index, row in df_branch.iterrows():
    L.append((index + 1, int(row["fbus"]), int(row["tbus"])))
# set of generators
G = []
for index, row in df_generator.iterrows():
    G.append((index + 1, int(row["bus"])))


# set of facilities
# w_d represents the weighting factor for each load and hour, initializing to 1
w_d = {}
for d in D_wh:
    if D_wh[d] == 0:
        w_d[d] = 0
    else:
        w_d[d] = random.choices(
            population=[1, 10, 20, 8, 15], weights=[0.1, 0.2, 0.2, 0.4, 0.1]
        )[0]


#################################################
####              Parameters                 ####
#################################################

# trade-off parameter
alpha = 0.5

# bus-area pair
bus_area = {}
for index, row in df_bus.iterrows():
    bus_area[int(row["bus_i"])] = int(row["area"])

# base risk of wildfire ignition in each area
R_j = {}
for area in A_e:
    R_j[area] = 1.1

# relative risk factor for electrical components in areas at each hour
kappa_ejh = {}
for area in A_e:
    kappa_ejh[("Bus", area)] = 1
    kappa_ejh[("Gen", area)] = 1
    kappa_ejh[("Line", area)] = 1
    kappa_ejh[("Load", area)] = 1


# meteorological factor at each hour
gamma_h = {
    1: 1.00,
    2: 1.00,
    3: 1.00,
    4: 1.00,
    5: 1.00,  # 01:00 AM - 05:00 AM
    6: 2,  # 06:00 AM
    7: 3,
    8: 4,  # 07:00 AM - 08:00 AM
    9: 5,  # 09:00 AM
    10: 6,
    11: 8,  # 10:00 AM - 11:00 AM
    12: 10,  # 12:00 PM
    13: 10.8,  # 01:00 PM
    14: 12,
    15: 12,  # 02:00 PM - 03:00 PM
    16: 10,  # 04:00 PM
    17: 8.5,  # 05:00 PM
    18: 7,  # 06:00 PM
    19: 5,  # 07:00 PM
    20: 4,
    21: 1.00,
    22: 1.00,
    23: 1.00,
    24: 1.00,  # 08:00 PM - 12:00 AM
}

# voltage angle theta, max theta and min theta
theta_max = {}
theta_min = {}

for index, row in df_branch.iterrows():
    theta_max[(index + 1, row["fbus"], row["tbus"])] = row["angmax"]
    theta_min[(index + 1, row["fbus"], row["tbus"])] = row["angmin"]

# risk calculations
R_d = {}
R_g = {}
R_l = {}
R_i = {}
for h in H:
    # R_d: load
    for b in B:
        R_d[b, h] = kappa_ejh[("Bus", bus_area[b])] * gamma_h[h] * R_j[bus_area[b]]

    # R_g: generator
    for g, b in G:
        R_g[(g, b), h] = kappa_ejh[("Gen", bus_area[b])] * gamma_h[h] * R_j[bus_area[b]]

    # R_l: line
    for index, i, j in L:
        R_l[(index, i, j), h] = (
            kappa_ejh[("Line", bus_area[i])] * gamma_h[h] * R_j[bus_area[i]]
            + kappa_ejh[("Line", bus_area[j])] * gamma_h[h] * R_j[bus_area[j]]
        )

    # R_i : bus
    for b in B:
        R_i[b, h] = kappa_ejh[("Bus", bus_area[b])] * gamma_h[h] * R_j[bus_area[b]]

# Typical load served during standard operational phases for each load and hour

Residential_ratio = {
    1: 1.02,
    2: 0.99,
    3: 0.95,
    4: 0.92,
    5: 0.89,
    6: 0.88,
    7: 0.89,
    8: 0.91,
    9: 0.93,
    10: 0.95,
    11: 0.97,
    12: 0.99,
    13: 1.01,
    14: 1.02,
    15: 1.03,
    16: 1.03,
    17: 1.04,
    18: 1.06,
    19: 1.07,
    20: 1.08,
    21: 1.08,
    22: 1.09,
    23: 1.07,
    24: 1.05,
}
Medical_ratio = {
    1: 0.94,
    2: 0.93,
    3: 0.91,
    4: 0.90,
    5: 0.88,
    6: 0.87,
    7: 0.87,
    8: 0.89,
    9: 0.95,
    10: 1.00,
    11: 1.02,
    12: 1.04,
    13: 1.08,
    14: 1.12,
    15: 1.14,
    16: 1.14,
    17: 1.14,
    18: 1.11,
    19: 1.08,
    20: 1.04,
    21: 1.02,
    22: 1.00,
    23: 0.99,
    24: 0.96,
}
Educational_ratio = {
    1: 0.86,
    2: 0.85,
    3: 0.84,
    4: 0.83,
    5: 0.84,
    6: 0.87,
    7: 0.91,
    8: 0.97,
    9: 1.02,
    10: 1.09,
    11: 1.13,
    12: 1.15,
    13: 1.16,
    14: 1.16,
    15: 1.16,
    16: 1.15,
    17: 1.12,
    18: 1.08,
    19: 1.03,
    20: 1.00,
    21: 0.98,
    22: 0.95,
    23: 0.92,
    24: 0.88,
}
Commercial_ratio = {
    1: 0.88,
    2: 0.79,
    3: 0.68,
    4: 0.67,
    5: 0.71,
    6: 0.64,
    7: 0.70,
    8: 0.94,
    9: 1.01,
    10: 1.06,
    11: 1.08,
    12: 1.12,
    13: 1.21,
    14: 1.27,
    15: 1.26,
    16: 1.25,
    17: 1.22,
    18: 1.22,
    19: 1.25,
    20: 1.17,
    21: 1.12,
    22: 1.01,
    23: 0.91,
    24: 0.90,
}

Administrative_ratio = {
    1: 0.86,
    2: 0.85,
    3: 0.84,
    4: 0.83,
    5: 0.82,
    6: 0.84,
    7: 0.88,
    8: 0.95,
    9: 1.01,
    10: 1.07,
    11: 1.11,
    12: 1.15,
    13: 1.16,
    14: 1.18,
    15: 1.19,
    16: 1.18,
    17: 1.16,
    18: 1.11,
    19: 1.05,
    20: 1.00,
    21: 0.98,
    22: 0.95,
    23: 0.92,
    24: 0.88,
}

# D_dh = {(d, h): D_wh[d] for d in D_wh for h in H}
D_dh = {}
for d in D_wh:
    if w_d[d] == 1:
        for h in H:
            D_dh[d, h] = D_wh[d] * Commercial_ratio[h]
    elif w_d[d] == 10:
        for h in H:
            D_dh[d, h] = D_wh[d] * Educational_ratio[h]
    elif w_d[d] == 20:
        for h in H:
            D_dh[d, h] = D_wh[d] * Medical_ratio[h]
    elif w_d[d] == 8:
        for h in H:
            D_dh[d, h] = D_wh[d] * Residential_ratio[h]
    else:
        for h in H:
            D_dh[d, h] = D_wh[d] * Administrative_ratio[h]

# Pmax and Pmin
P_max = {}
P_min = {}

for index, row in df_generator.iterrows():
    P_max[(index + 1, int(row["bus"]))] = row["Pmax"]
    P_min[(index + 1, int(row["bus"]))] = row["Pmin"]
# define thermal and susceptance values
T_l = {}  # Example thermal limits
for index, row in df_branch.iterrows():
    if row["thermal"] == 0:
        T_l[(index + 1, row["fbus"], row["tbus"])] = 1000000
    else:
        T_l[(index + 1, row["fbus"], row["tbus"])] = row["thermal"]
b_l = {}  # Example susceptance values
for index, row in df_branch.iterrows():
    b_l[(index + 1, row["fbus"], row["tbus"])] = row["susceptance"]

In [11]:
def hour_based_wildfire_model(
    h,
    alpha,
    D_wh,
    A_e,
    G,
    L,
    B,
    H,
    P_min,
    P_max,
    T_l,
    b_l,
    theta_max,
    theta_min,
    R_d,
    R_g,
    R_l,
    R_i,
):
    model = gp.Model("Power System and Wildfire Risk Management", env=env)

    #################################################
    ####           Decision Variables            ####
    #################################################

    # indicate whether load l will remain energized during hour h
    z_lh = model.addVars(L, H, vtype=GRB.BINARY, name="z_lh")

    # indicate whether bus i will remain energized during hour h
    z_ih = model.addVars(B, H, vtype=GRB.BINARY, name="z_ih")

    # indicate whether generator l will remain energized during hour h
    z_gh = model.addVars(G, H, vtype=GRB.BINARY, name="z_gh")

    # represent the fraction of the load that that is de-energized during hour h
    x_dh = model.addVars(D_wh, H, vtype=GRB.CONTINUOUS, name="x_dh", lb=0, ub=1)

    # represent the power generation of generators in hour h
    P_gh = model.addVars(G, H, lb=0, name="P_gh")

    # represent the power flows on transmission lines l in L from bus j to bus i in hour h
    P_l_ij_h = model.addVars(L, H, lb=-GRB.INFINITY, name="P_l_h")

    # represent the voltage angles
    theta_ih = model.addVars(B, H, lb=-180, ub=180, name="theta_ih")

    # Total wildfire risk calculation
    R_Fire_h = {
        h: sum(x_dh[b, h] * R_d[b, h] for b in D_wh)
        + sum(z_gh[g, b, h] * R_g[(g, b), h] for (g, b) in G)
        + sum(z_lh[l, fb, tb, h] * R_l[(l, fb, tb), h] for (l, fb, tb) in L)
        + sum(z_ih[b, h] * R_i[b, h] for b in B)
        for h in H
    }
    # Calculate the total load successfully distributed across the network within hour h
    # D_Tot_h = {h: sum( 1 for d in D_wh) for h in H}
    D_Tot_h = {h: sum(x_dh[d, h] * w_d[d] * D_dh[d, h] for d in D_wh) for h in H}
    # D_Tot_h = {h: sum(x_dh[d, h] * D_dh[d, h] for d in D_wh) for h in H}

    #################################################
    ####           Objective Function            ####
    #################################################

    model.setObjective(
        (1 - alpha) * D_Tot_h[h] - alpha * R_Fire_h[h],
        GRB.MAXIMIZE,
    )
    #################################################
    ####               Constraints               ####
    #################################################

    # Energization constraints
    for i in B:
        # Loads must be less than or equal to bus energization status
        # 7a
        model.addConstr(
            (x_dh[i, h] <= z_ih[i, h]),
            name=f"load_energization_{i}_{h}",
        )
        # Generator status must be less than or equal to bus energization status
        # 7b
        model.addConstrs(
            (z_gh[g, b, h] <= z_ih[i, h] for (g, b) in G if i == b),
            name=f"gen_energization_{i}_{h}",
        )
        # Line energization
        # 7c
        model.addConstrs(
            (z_lh[l, a, b, h] <= z_ih[i, h] for (l, a, b) in L if i == a or i == b),
            name=f"line_energization_{i}_{h}",
        )

    # Generation constraints
    # 8
    for g, b in G:
        model.addConstr(
            P_min[(g, b)] * z_gh[g, b, h] <= P_gh[g, b, h], name=f"Pmin_{g}_{b}_{h}"
        )
        model.addConstr(
            P_gh[g, b, h] <= P_max[(g, b)] * z_gh[g, b, h], name=f"Pmax_{g}_{b}_{h}"
        )

    # Power flow and node balance constraints
    for l, i, j in L:
        # 9a
        model.addConstr(
            P_l_ij_h[l, i, j, h]
            <= -b_l[(l,i, j)]
            * (
                theta_ih[i, h]
                - theta_ih[j, h]
                + theta_max[(l, i, j)] * (1 - z_lh[l, i, j, h])
            ),
            name=f"angle_limit_upper_{l}_{i}_{j}_{h}",
        )

        # 9b
        model.addConstr(
            P_l_ij_h[l, i, j, h]
            >= -b_l[(l,i, j)]
            * (
                theta_ih[i, h]
                - theta_ih[j, h]
                + theta_min[(l, i, j)] * (1 - z_lh[l, i, j, h])
            ),
            name=f"angle_limit_lower_{l}_{i}_{j}_{h}",
        )

        # 9c
        model.addConstr(
            -T_l[(l,i, j)] * z_lh[l, i, j, h] <= P_l_ij_h[l, i, j, h],
            name=f"flow_limit_lower_{l}_{i}_{j}_{h}",
        )
        model.addConstr(
            P_l_ij_h[l, i, j, h] <= T_l[(l,i, j)] * z_lh[l, i, j, h],
            name=f"flow_limit_upper_{l}_{i}_{j}_{h}",
        )

    # Power flow and node balance constraints
    # 9d
    for i in B:
        model.addConstr(
            sum(P_gh[g, b, h] for (g, b) in G if i == b)
            - sum(P_l_ij_h[l, a, b, h] for (l, a, b) in L if i == a)
            + sum(P_l_ij_h[l, a, b, h] for (l, a, b) in L if i == b)
            - x_dh[i, h] * D_wh[i]
            == 0,
            name=f"node_balance_{i}_{h}",
        )

    # Solve the model
    model.optimize()

    # print result
    # Check if the optimal solution is found
    
    if model.status == GRB.OPTIMAL:
        print("Optimal Solution:")
        print("Optimal Value = ",model.ObjVal)

        # Print operational status for generators, lines, and buses
        # for h in H:
        print(f"Hour {h}:")
        for g, b in G:
            print(
                f"  Generator {g}: Energized = {'Yes' if z_gh[g,b, h].X > 0.5 else 'No'}, Power Generated = {P_gh[g,b, h].X:.2f} MW"
            )
        for l, fb, tb in L:
            print(
                f"  Line {l}_{fb}_{tb}: Energized = {'Yes' if z_lh[l,fb,tb, h].X > 0.5 else 'No'}"
            )
        for b in B:
            print(f"  Bus {b}: Energized = {'Yes' if z_ih[b, h].X > 0.5 else 'No'}")

        # Print load shedding details
        for d in D_wh:
            print(f"  Load {d} Shedding: {x_dh[d, h].X * 100:.2f}% of {D_wh[d]} MW")

        # Print power flows on transmission lines if needed
        for l, fb, tb in L:
            if P_l_ij_h[l, fb, tb, h].X != 0:
                print(
                    f"  Power Flow on Line {l} from Bus {fb} to Bus {tb}: {P_l_ij_h[l,fb,tb, h].X:.2f} MW"
                )

        # Print voltage angles at buses if needed
        for b in B:
            print(f"  Voltage Angle at Bus {b}: {theta_ih[b, h].X:.2f} degrees")
        print(f"\n")
    else:
        print("No optimal solution found.")
    

In [12]:
# hour 1
hour_based_wildfire_model(
        h=1,
        alpha=alpha,
        D_wh=D_wh,
        A_e=A_e,
        G=G,
        L=L,
        B=B,
        H=H,
        P_min=P_min,
        P_max=P_max,
        T_l=T_l,
        b_l=b_l,
        theta_max=theta_max,
        theta_min=theta_min,
        R_d=R_d,
        R_g=R_g,
        R_l=R_l,
        R_i=R_i,
    )

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.6.0 22G90)

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 324 rows, 4416 columns and 662 nonzeros
Model fingerprint: 0xf10dd19d
Variable types: 2568 continuous, 1848 integer (1848 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [4e+00, 1e+01]
Found heuristic solution: objective -14.8500000
Presolve removed 316 rows and 4405 columns
Presolve time: 0.01s
Presolved: 8 rows, 11 columns, 25 nonzeros
Found heuristic solution: objective 13.5800000
Variable types: 9 continuous, 2 integer (2 binary)

Root relaxation: objective 1.613696e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   

In [13]:
for h in H:
    hour_based_wildfire_model(
        h=h,
        alpha=alpha,
        D_wh=D_wh,
        A_e=A_e,
        G=G,
        L=L,
        B=B,
        H=H,
        P_min=P_min,
        P_max=P_max,
        T_l=T_l,
        b_l=b_l,
        theta_max=theta_max,
        theta_min=theta_min,
        R_d=R_d,
        R_g=R_g,
        R_l=R_l,
        R_i=R_i,
    )

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[x86] - Darwin 22.6.0 22G90)



CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 324 rows, 4416 columns and 662 nonzeros
Model fingerprint: 0xf10dd19d
Variable types: 2568 continuous, 1848 integer (1848 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+02]
  Objective range  [5e-01, 3e+02]
  Bounds range     [1e+00, 2e+02]
  RHS range        [4e+00, 1e+01]
Found heuristic solution: objective -14.8500000
Presolve removed 316 rows and 4405 columns
Presolve time: 0.01s
Presolved: 8 rows, 11 columns, 25 nonzeros
Found heuristic solution: objective 13.5800000
Variable types: 9 continuous, 2 integer (2 binary)

Root relaxation: objective 1.613696e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  161.36959    0    1   13.58000  161.36959  1088%  